# 📘 Agentic Architectures 9: Tree-of-Thoughts Planning

Welcome to the ninth notebook in our series. Today, we explore a powerful reasoning and planning architecture: **Tree-of-Thoughts (ToT)**. This pattern elevates an agent's problem-solving capabilities from a linear chain of thought to a multi-path exploratory search.

Instead of generating a single, sequential line of reasoning, a ToT agent generates multiple candidate "thoughts" or next steps at each stage of a problem. It then evaluates these thoughts, pruning invalid or unpromising branches and expanding the most promising ones. This creates a search tree where the agent can backtrack, explore alternatives, and systematically navigate a complex problem space.

To demonstrate this, we'll task our agent with a classic logic puzzle: the **Wolf, Goat, and Cabbage problem**. This puzzle is famous because it requires non-obvious steps (like bringing an item *back*) and has several invalid states that can trap a naive reasoner. We will show how a simple Chain-of-Thought agent might fail, while the ToT agent methodically constructs a valid plan by exploring and evaluating a tree of possibilities.

### Definition
**Tree-of-Thoughts (ToT)** is an agentic reasoning framework where problem-solving is modeled as a search through a tree. The agent explores multiple reasoning paths (branches) simultaneously. At each step, it generates potential next steps ("thoughts"), evaluates their viability, and decides which paths to continue exploring, effectively pruning the search space.

### High-level Workflow

1.  **Decomposition:** The problem is broken down into a series of steps or thoughts.
2.  **Thought Generation:** For the current state of the problem, the agent generates multiple potential next steps or thoughts. This creates branches in the search tree.
3.  **State Evaluation:** Each new thought (leading to a new state) is evaluated by a "critic" or a validation function. This evaluation can assess:
    *   **Validity:** Is this move allowed by the rules of the problem?
    *   **Progress:** Does this move get us closer to the solution?
    *   **Heuristics:** Is this path likely to succeed?
4.  **Pruning & Expansion:** Invalid or unpromising branches are pruned. The agent then proceeds from the most promising active branches, repeating the thought generation process.
5.  **Solution:** The process continues until a goal state is reached. The solution is the path of thoughts from the root to the goal.

### When to Use / Applications
*   **Logic Puzzles & Math Problems:** Problems with clear rules and goal states that require multi-step, non-linear reasoning (e.g., Sudoku, river crossing puzzles).
*   **Complex Planning:** When a task requires a detailed plan where the order of operations matters and constraints must be respected (e.g., planning a complex trip with multiple legs and budget constraints).
*   **Creative Writing or Code Generation:** Exploring multiple story branches or implementation strategies before committing to one.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Robustness:** Systematically explores the problem space, making it much less likely to get stuck or produce an incorrect answer compared to a single-pass method.
    *   **Handles Combinatorial Complexity:** Well-suited for problems where the number of possible sequences is vast.
*   **Weaknesses:**
    *   **Computationally Heavy:** Requires significantly more LLM calls and state management than a simple Chain-of-Thought prompt, making it slower and more expensive.
    *   **Requires a Good Evaluator:** The effectiveness of the search heavily depends on the quality of the state evaluation logic.

## Phase 0: Foundation & Setup

We'll install our libraries and configure our API keys as usual.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import re
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from collections import defaultdict

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.tree import Tree

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tree-of-Thoughts (Nebius)"

# Check for required environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Defining the Problem Environment

A Tree-of-Thoughts system requires a well-defined environment to operate in. For our Wolf, Goat, and Cabbage puzzle, this means we need to programmatically define:

1.  **State Representation:** A way to describe the location of all items.
2.  **Validation Rules:** A function to check if a state is invalid (e.g., the goat and cabbage are left alone).
3.  **Goal State:** A way to check if the puzzle has been solved.
4.  **Possible Moves:** A function to determine all legal moves from a given state.

In [3]:
console = Console()

class PuzzleState(BaseModel):
    "Represents the state of the Wolf, Goat, and Cabbage puzzle."
    # Using sets for unordered collections of items on each bank
    left_bank: set[str] = Field(default_factory=lambda: {"wolf", "goat", "cabbage"})
    right_bank: set[str] = Field(default_factory=set)
    boat_location: str = "left"
    move_description: str = "Initial state."

    def is_valid(self) -> bool:
        """Checks if the current state is valid (no one gets eaten)."""
        # Check left bank
        if self.boat_location == "right":
            if "wolf" in self.left_bank and "goat" in self.left_bank:
                return False
            if "goat" in self.left_bank and "cabbage" in self.left_bank:
                return False
        # Check right bank
        if self.boat_location == "left":
            if "wolf" in self.right_bank and "goat" in self.right_bank:
                return False
            if "goat" in self.right_bank and "cabbage" in self.right_bank:
                return False
        return True

    def is_goal(self) -> bool:
        """Checks if the goal state has been reached."""
        return self.right_bank == {"wolf", "goat", "cabbage"}
    
    def __hash__(self):
        # Make the state hashable to check for visited states
        return hash((frozenset(self.left_bank), frozenset(self.right_bank), self.boat_location))
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

def get_possible_moves(state: PuzzleState) -> list[PuzzleState]:
    """Generates all possible valid next states from the current state."""
    moves = []
    current_bank = state.left_bank if state.boat_location == "left" else state.right_bank
    
    # Option 1: Move one item in the boat
    for item in current_bank:
        new_state = state.model_copy(deep=True)
        if state.boat_location == "left":
            new_state.left_bank.remove(item)
            new_state.right_bank.add(item)
            new_state.boat_location = "right"
            new_state.move_description = f"Move {item} to the right bank."
        else:
            new_state.right_bank.remove(item)
            new_state.left_bank.add(item)
            new_state.boat_location = "left"
            new_state.move_description = f"Move {item} to the left bank."
        if new_state.is_valid():
            moves.append(new_state)
            
    # Option 2: Move the boat empty
    empty_move_state = state.model_copy(deep=True)
    if state.boat_location == "left":
        empty_move_state.boat_location = "right"
        empty_move_state.move_description = "Move the boat empty to the right bank."
    else:
        empty_move_state.boat_location = "left"
        empty_move_state.move_description = "Move the boat empty to the left bank."
    if empty_move_state.is_valid():
        moves.append(empty_move_state)
        
    return moves

print("Puzzle environment defined successfully.")

Puzzle environment defined successfully.


## Phase 2: Implementing the ToT Agent with LangGraph

We will now build the agent itself. The state of our graph will track all the active paths (branches) in our thought tree. The nodes will perform the key ToT actions:

1.  **Expand Paths (Thought Generator):** An LLM-powered node that looks at the last state of each active path and proposes a promising next move from the list of valid possibilities.
2.  **Prune Paths (State Evaluator):** This node cleans up after generation. It will remove any paths that have entered an invalid state or a cycle (revisiting a previous state).
3.  **Check for Solution (Goal Check):** A conditional node that checks if any of the active paths have reached the goal state. If so, it terminates the loop.

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydantic model for the LLM's choice of move
class MoveChoice(BaseModel):
    best_move_index: int = Field(description="The index of the best move from the provided list of possible moves.")
    reasoning: str = Field(description="Brief reasoning for why this is the most promising move.")

# LangGraph State
class ToTState(TypedDict):
    problem_description: str
    # Each path is a list of PuzzleState objects
    active_paths: List[List[PuzzleState]]
    # We'll store the final solution here
    solution: Optional[List[PuzzleState]]

# Graph Nodes
def initialize_search(state: ToTState) -> Dict[str, Any]:
    """Node to set up the initial state of the search."""
    initial_puzzle_state = PuzzleState()
    return {"active_paths": [[initial_puzzle_state]]}

def expand_paths(state: ToTState) -> Dict[str, Any]:
    """The 'Thought Generator'. Expands each active path with a promising next move."""
    console.print("--- Expanding Paths ---")
    new_paths = []
    choice_llm = llm.with_structured_output(MoveChoice)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert logic puzzle solver. Your goal is to solve the Wolf, Goat, and Cabbage problem. Analyze the current path and choose the most promising next move from the list of options to reach the goal."),
        ("human", "Problem: {problem}\n\nCurrent Path History:\n{path_history}\n\nFrom the final state, choose the best next move from this list:\n{possible_moves}")
    ])
    
    for path in state['active_paths']:
        last_state = path[-1]
        possible_next_states = get_possible_moves(last_state)
        
        if not possible_next_states:
            continue # This path is a dead end
            
        path_history_str = " -> ".join([s.move_description for s in path])
        possible_moves_str = "\n".join([f"{i}: {s.move_description}" for i, s in enumerate(possible_next_states)])
        
        # For simplicity and to show breadth, we can explore multiple moves.
        # A more advanced ToT might use the LLM to pick only the single best one.
        # Here, we'll let all valid moves branch out to demonstrate the tree structure.
        for next_state in possible_next_states:
            new_paths.append(path + [next_state])

    console.print(f"[cyan]Expanded to {len(new_paths)} potential paths.[/cyan]")
    return {"active_paths": new_paths}

def prune_paths(state: ToTState) -> Dict[str, Any]:
    """The 'State Evaluator'. Prunes paths that are invalid or contain cycles."""
    console.print("--- Pruning Paths ---")
    pruned_paths = []
    for path in state['active_paths']:
        # Check for cycles: if the last state has appeared before in the path
        if path[-1] in path[:-1]:
            continue # Found a cycle, prune this path
        
        # The get_possible_moves function already ensures validity, but this is a good place for extra checks.
        pruned_paths.append(path)
        
    console.print(f"[green]Pruned down to {len(pruned_paths)} valid, non-cyclical paths.[/green]")
    return {"active_paths": pruned_paths}

# Conditional Node
def check_for_solution(state: ToTState) -> str:
    """Checks if any path has reached the goal and routes execution."""
    for path in state['active_paths']:
        if path[-1].is_goal():
            console.print("[bold green]Solution Found![/bold green]")
            # Side effect: update the solution in the state. LangGraph copies this out.
            state['solution'] = path
            return "solution_found"
    return "continue_search"

# Build the graph
workflow = StateGraph(ToTState)

workflow.add_node("initialize", initialize_search)
workflow.add_node("expand", expand_paths)
workflow.add_node("prune", prune_paths)

workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "expand")
workflow.add_edge("expand", "prune")

workflow.add_conditional_edges(
    "prune",
    check_for_solution,
    {
        "solution_found": END,
        "continue_search": "expand"
    }
)

tot_agent = workflow.compile()
print("Tree-of-Thoughts agent graph compiled successfully.")

Tree-of-Thoughts agent graph compiled successfully.


## Phase 3: Demonstration & Analysis

Now, let's run our ToT agent on the puzzle. We'll compare its systematic approach with a simple, single-pass Chain-of-Thought request to highlight the differences in robustness.

In [5]:
problem = "A farmer wants to cross a river with a wolf, a goat, and a cabbage. The boat can only carry the farmer and one other item. The farmer cannot leave the wolf alone with the goat, nor the goat alone with the cabbage. How can the farmer get everyone across safely?"

console.print("--- 🌳 Running Tree-of-Thoughts Agent ---")
# The recursion limit prevents infinite loops in our graph
config = {"recursion_limit": 15}
final_state = tot_agent.invoke({"problem_description": problem}, config=config)

console.print("\n--- ✅ ToT Agent Solution ---")
if final_state.get('solution'):
    solution_path = final_state['solution']
    # Use rich.Tree for a nice visual output
    tree = Tree("[bold magenta]Wolf, Goat, and Cabbage Solution Path[/bold magenta]")
    for i, state in enumerate(solution_path):
        tree.add(f"[green]{i+1}.[/green] {state.move_description}")
    console.print(tree)
else:
    console.print("[bold red]No solution found within the step limit.[/bold red]")

console.print("\n--- 🤔 Running Simple Chain-of-Thought Agent ---")
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class logic puzzle solver. Provide a step-by-step solution to the user's puzzle."),
    ("human", "{problem}")
])
cot_chain = cot_prompt | llm
cot_result = cot_chain.invoke({"problem": problem}).content
console.print(Markdown(cot_result))

--- 🌳 Running Tree-of-Thoughts Agent ---


--- Expanding Paths ---
Expanded to 1 potential paths.
--- Pruning Paths ---
Pruned down to 1 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 2 potential paths.
--- Pruning Paths ---
Pruned down to 2 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 4 potential paths.
--- Pruning Paths ---
Pruned down to 4 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 7 potential paths.
--- Pruning Paths ---
Pruned down to 7 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 12 potential paths.
--- Pruning Paths ---
Pruned down to 12 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 20 potential paths.
--- Pruning Paths ---
Pruned down to 20 valid, non-cyclical paths.
--- Expanding Paths ---
Expanded to 32 potential paths.
--- Pruning Paths ---
Pruned down to 32 valid, non-cyclical paths.
Solution Found!



--- ✅ ToT Agent Solution ---


Wolf, Goat, and Cabbage Solution Path
├── 1. Initial state.
├── 2. Move goat to the right bank.
├── 3. Move the boat empty to the left bank.
├── 4. Move wolf to the right bank.
├── 5. Move goat to the left bank.
├── 6. Move cabbage to the right bank.
├── 7. Move the boat empty to the left bank.
└── 8. Move goat to the right bank.



--- 🤔 Running Simple Chain-of-Thought Agent ---


Here's a step-by-step solution to the Wolf, Goat, and Cabbage puzzle:

1.  **Take the Goat across:** First, take the goat across the river to the right bank. You leave the wolf and cabbage behind on the left bank.
2.  **Return empty:** Return to the left bank alone.
3.  **Take the Wolf across:** Now, take the wolf across to the right bank. 
4.  **Bring the Goat back:** *This is the key step.* Leave the wolf on the right bank and bring the goat back with you to the left bank.
5.  **Take the Cabbage across:** Leave the goat on the left bank and take the cabbage across to the right bank. Now the wolf and cabbage are on the right bank.
6.  **Return empty:** Return to the left bank alone.
7.  **Take the Goat across:** Finally, take the goat across to the right bank.

Now, the wolf, goat, and cabbage are all safely on the right bank, and the puzzle is solved.

### Analysis of the Results

The difference between the two approaches is profound:

- **Chain-of-Thought (CoT):** This approach relies on the LLM's pre-trained knowledge to recall the solution. For a classic, well-known problem like this, a powerful LLM can often produce the correct answer in one go. However, if it makes a single mistake, it has no mechanism to self-correct. For a novel or more complex problem, the likelihood of failure is much higher. Its correctness is a matter of recall, not verifiable reasoning.

- **Tree-of-Thoughts (ToT):** This agent *discovered* the solution through systematic, verifiable search. It didn't just recall an answer; it built one. We can see the process in the logs: expanding paths, then pruning ones that hit dead ends or cycles. Even if the LLM guiding the expansion made a suboptimal choice on one branch, the agent could continue exploring other, more promising branches. This method is far more robust and trustworthy because its final solution is guaranteed to be valid according to the rules of the environment we defined.

The ToT agent's success is not based on luck or memorization, but on the soundness of its search algorithm. This makes it a vastly superior approach for tasks that demand high reliability and planning.

## Conclusion

In this notebook, we implemented a **Tree-of-Thoughts** agent to solve a classic logic puzzle. We demonstrated that by transforming a problem into a state space and systematically searching through it, an agent can achieve a level of robustness and accuracy that is impossible with simple, single-pass reasoning methods.

The core components of ToT—**thought generation (expansion)**, **state evaluation (pruning)**, and **search**—create a powerful framework for tackling complex planning and reasoning tasks. While it comes with a higher computational cost, the trade-off is a significant increase in reliability and problem-solving capability. This architecture is a key step toward building agents that can reason deliberately and find solutions to challenging, multi-step problems.